In [1]:
import cv2
from glob import glob
from ultralytics import YOLO
from tqdm import tqdm
import torchvision.models as models
from typing import List

import torch.nn as nn
import torch
import os
import math
import csv

## 이전 파일 클래스 정의

In [8]:
from torchvision.models import resnet50
class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.base_model = resnet50(pretrained=True)
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [2]:
class TwoPointCoordinate:
    def __init__(self, start_point: tuple, end_point: tuple):
        self.start_point = start_point
        self.end_point = end_point

    def get_xy_coords(self):
        # x1 = x4, x2 = x3
        # y1 = y2, y3 = y4
        x1, y1 = self.start_point[1], self.start_point[0]
        x3, y3 = self.end_point[1], self.end_point[0]
        return [x1, y1, x3, y1, x3, y3, x1, y3]

class YoloCoordinate:
    def __init__(self, w, h, center_x, center_y):
        self.w = w
        self.h = h
        self.center_x = center_x
        self.center_y = center_y

    def convert_two_point_coordinate(self) -> TwoPointCoordinate:
        half_width = self.w / 2
        half_height = self.h / 2
        img_width = 1920
        img_height = 1040

        point1_x = self.center_x - half_width # box의 왼쪽 위 꼭지점
        point1_y = self.center_y - half_height # box의 왼쪽 위 꼭지점

        point3_x = self.center_x + half_width # box의 오른쪽 아래 꼭지점
        point3_y = self.center_y + half_height # box의 오른쪽 아래 꼭지점

        point1_x = point1_x * img_width
        point1_y = point1_y * img_height

        point3_x = point3_x * img_width
        point3_y = point3_y * img_height

        return TwoPointCoordinate((point1_y, point1_x), (point3_y, point3_x))

In [3]:
# 파일 이름, 라벨, 좌표 클래스 (TwoPoints.., Yolo..)를 담는 클래스
class ImageContainer:
    def __init__(self, file_names : List[str], labels: List[str], coordinates: List):
        self.coordinates = coordinates
        self.file_names = file_names
        self.file_paths = []
        self.labels = labels
        self.convert_coordinates()

    def convert_coordinates(self):
        if isinstance(self.coordinates[0], YoloCoordinate):
            for idx, coordinate in tqdm(enumerate(self.coordinates), desc='TwoPointCoordinate로 변환'):
                self.coordinates[idx] = coordinate.convert_two_point_coordinate()

    def get_file_paths(self, path):
        for idx, name in enumerate(self.file_names):
            self.file_paths.append(path + name)
        return self.file_paths

In [4]:
# 이미지 컨테이너에 담긴 이미지 정보를 토대로 좌표에 맞추어 자르고 저장하는 클래스
class LightImageCropper: # 자를려면 자를 이미지, 이미지 좌표
    def __init__(self, container: ImageContainer):
        self.container = container
        self.images = []
        self.cropped_images = []
        self.file_num = 0

    def cr(self):
        coordinates = self.container.coordinates

        self.load_images(self.container.file_names)
        self.crop(coordinates)

        return self.cropped_images

    def crop(self, coordinates):
        for idx, coordinate in tqdm(enumerate(coordinates), leave=False):
            y_1 = int(coordinate.start_point[0])
            x_1 = int(coordinate.start_point[1])
            y_2 = int(coordinate.end_point[0])
            x_2 = int(coordinate.end_point[1])
            self.cropped_images.append(self.images[idx][y_1: y_2, x_1: x_2, :].copy())

    def load_images(self, paths): # 이미지 파일 폴더 경로 입력
        for path in paths:
            image = cv2.imread(path, cv2.IMREAD_COLOR)
            if image is None:
                raise Exception("이미지가 없습니다. 경로확인")

            self.images.append(image)



## 신규 클래스 ModelParser
모델을 입력받아서 ImageContainer로 저장하는 클래스

In [5]:

class ModelParser:
    def __init__(self, model, img_paths):
        self.model = model
        self.img_paths = img_paths
        self.detections = { "names": [], "labels": [], "boxes": [], "confs": []}

    def parse(self) -> ImageContainer:
        count = 0
        for path in self.img_paths:
            img = cv2.imread(path, cv2.IMREAD_COLOR)

            if self.__img_validation(img) or self.__coord_validation(img):
                self.detect(img, path)
            else: count += 1

        print(f"{count}개의 이미지가 걸려졌어요")
        return ImageContainer(self.detections["names"], self.detections["labels"], self.detections["boxes"])

    @staticmethod
    def __img_validation(img):
        height, width, _ = img.shape
        return height >= 80 or width >= 150

    # 동민님 여기에 메서드 채워넣으시면 됩니다!!!!!
    @staticmethod
    def __coord_validation(img):
        # 메서드 작성 .....
        return True

    def detect(self, img, path): # 욜로 모델에서 이미지를 불러와서 탐지한 후 json으로 변환
        detections = self.model(img)[0]
        conf_li = detections.boxes.conf.tolist()
        self.detections["labels"] += detections.boxes.cls.int().tolist()
        self.detections["boxes"] += self.__parse_coordinates(detections.boxes.xyxy.tolist())
        self.detections["confs"] += conf_li
        self.detections["names"] += [path for _ in range(len(conf_li))]
        self.__validation()

    def __parse_coordinates(self, coordinates):
        coord_li = []
        for coor in coordinates:
            coord_li.append(self.__trans_coordinate(coor))
        return coord_li

    def __trans_coordinate(self, coordinate):
        point1_y, point1_x = coordinate[1], coordinate[0]
        point3_y, point3_x = coordinate[3], coordinate[2]
        start_point, end_point = (point1_y, point1_x), (point3_y, point3_x)
        return TwoPointCoordinate(start_point, end_point)

    def __validation(self):
        labels, boxes, confs = self.detections["labels"], self.detections["boxes"], self.detections["confs"]
        if len(labels) != len(boxes) != confs:
            raise Exception("안맞는데요?")



In [6]:
def calc_batch_range(process, container_len, batch_size):
    if batch_size + process > container_len:
        return process, container_len - 1
    else:
        return process, process + batch_size

def flatten(lst):
    result = []
    for i in lst:
        if isinstance(i, list):
            result.extend(flatten(i))
        else:
            result.append(i)
    return result

## ImageClassifier, CsvWriter
ResNet 모델을 입력받아서 잘린 이미지를 분류하는 클래스
Csv파일로 변환하는 클래스

In [21]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor()
])



# 모델 예측

class ImageClassifier:
    def __init__(self, model, images):
        self.model = model
        self.images = images
        self.labels = []

    def classify(self, cutting_image):
        with torch.no_grad():
            self.model.eval()
            output = self.model(cutting_image.unsqueeze(0))  # 배치 차원 추가
            _, predicted_class = torch.max(output, 1)
        return predicted_class.item()


    def run_classifier(self): # 잘린 이미지들을 받아서 레스넷 돌리는 함수
        for image in tqdm(self.images, leave=False, position=0, desc='Processing images'):
            # OpenCV는 BGR 형식으로 이미지를 불러오므로 RGB 형식으로 변환
            img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # PIL Image로 변환
            img = Image.fromarray(img)
            img = transform(img)

            label = self.classify(img)
            self.labels.append(label)
        return self.labels

class CsvWriter:
    def __init__(self, file_names, labels, confidences, boxes):
        self.file_names = file_names
        self.labels = labels
        self.confidences = confidences
        self.boxes = boxes

    def run_writer(self, file_path="data.csv"):
        result = list(zip(self.file_names, self.labels, self.confidences, self.boxes))
        flatterned = [flatten(sublist) for sublist in result]
        self.write(flatterned, file_path)

    def write(self, data_list, filename="data.csv"):
        headers = [
            "file_name", "class_id", "confidence",
            "point1_x", "point1_y", "point2_x", "point2_y",
            "point3_x", "point3_y", "point4_x", "point4_y"
        ]

        # CSV 파일에 데이터 추가
        with open(filename, "a", newline="") as file:
            writer = csv.writer(file)

            # 헤더 작성
            if file.tell() == 0:
                writer.writerow(headers)

            # 데이터 작성
            for data in data_list:
                writer.writerow(data)
            print(f"{len(data_list)}개의 데이터가 저장되었습니다.")

## 경로변수 설정

In [32]:
# 경로 변수
model_path = f'./model/yolo/best.pt'  # best.pt 파일이 있는 경로
model = YOLO(model_path)
test_image_paths = glob("../detection/datasets/test/images/*")
# resNetModel = CustomResNet()
# model_state_dict = torch.load("./model/resnet.pt")
# resNetModel.load_state_dict(model_state_dict)

resNetModel = ResNet50(34)
resNetModel.load_state_dict(torch.load("./resnet50.pt"))

C:\Users\Parkjunho\anaconda3\envs\gluon\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Parkjunho\anaconda3\envs\gluon\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [33]:
len(test_image_paths)

3400

In [24]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# RUN!!

배치 사이즈는 메모리 상태보고 조절하시면 됩니다 32GB기준으로 1000정도가 적당합니다

In [34]:
# 메모리 상태 보시고 적절히 조절해주세요
batch_size = 500
process = 0
trials = math.ceil(len(test_image_paths) / batch_size)


for _ in tqdm(range(trials)):
    # 배치 사이즈 크기만큼 자르기
    start, end = calc_batch_range(process, len(test_image_paths), batch_size)

    # 모델에서 좌표값 읽어오기
    # ModelParser에서 이미지를 걸러냅니다
    print("모델에서 좌표값 읽어오기")
    model_parser = ModelParser(model, test_image_paths[start: end])
    container: ImageContainer = model_parser.parse()

    # 이미지 자르기
    print("이미지 자르기")

    cropper = LightImageCropper(container)
    cutting_images = cropper.cr()

    # 이미지 분류하기
    print("이미지 분류하기")

    labels = ImageClassifier(resNetModel, cutting_images).run_classifier()

    # csv파일 저장할 데이터 모으기
    print("csv파일 저장할 데이터 모으기")

    file_names = [ os.path.basename(file_path) for file_path in container.file_names]
    confidences = model_parser.detections["confs"]
    boxes = [ box.get_xy_coords() for box in model_parser.detections["boxes"] ]

    # csv파일 저장하기
    print("csv파일 저장하기")

    CsvWriter(file_names, labels, confidences, boxes).run_writer()
    process += batch_size


  0%|          | 0/7 [00:00<?, ?it/s]

모델에서 좌표값 읽어오기



0: 384x640 1 genesis_gv80_suv_2020_, 26.7ms
Speed: 3.1ms preprocess, 26.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 genesis_gv80_suv_2020_, 1 kia_carnival_van_2015_2020, 6.6ms
Speed: 2.0ms preprocess, 6.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_trailblazer_suv_2021_, 1 genesis_gv80_suv_2020_, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_ioniq_hatchback_2016_2019, 8.5ms
Speed: 1.2ms preprocess, 8.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_avante_sedan_2011_2015, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 kia_carnival_van_2015_2020, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 ssangyong_tivoli_suv_2016_2020, 6.6ms
Speed: 2.0ms p

0개의 이미지가 걸려졌어요
이미지 자르기



0it [00:00, ?it/s]
                  

...

0개의 이미지가 걸려졌어요
이미지 자르기



0it [00:00, ?it/s]
                  

이미지 분류하기


100%|██████████| 7/7 [28:45<00:00, 246.56s/it]                      

csv파일 저장할 데이터 모으기
csv파일 저장하기
809개의 데이터가 저장되었습니다.
